# Setup

In [1]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

Enter API key for OpenAI:  ········


# Tools

In [2]:
from langchain_core.tools import tool

from typing import Annotated
import subprocess
import tempfile


@tool
def compile_c(code: Annotated[str, "ANSI-C source code"]):
    """
    Compile C source code, returning the error message
    or 'success' if the program compiled successfully.
    """

    try:
        subprocess.run(["gcc", "-x", "c", "-"], input=code.encode(), capture_output=True, check=True)
        return "Compilation succeeded!"
    except subprocess.CalledProcessError as e:
        return f"Compilation failed!\n\n{e.stderr.decode()}"


@tool
def compile_latex(source: Annotated[str, "LaTeX source code"]) -> str:
    """
    Compile LaTeX source code in an environment with
    all texlive packages. Returns any errors produced,
    and whether the compilation succeeds.
    """

    with tempfile.TemporaryDirectory() as working_dir:
        source_path = os.path.join(working_dir, "input.tex")
        with open(source_path, "w") as source_file:
            source_file.write(source)

        output_name = "output"

        try:
            subprocess.run(
                [
                    "pdflatex",
                    f"-jobname={output_name}",
                    "-interaction=nonstopmode",
                    source_path,
                ],
                cwd=working_dir,
                capture_output = True,
                check = True
            )
            return "Compilation succeeded!"
        except subprocess.CalledProcessError as e:
            return f"Compilation failed!\n\n{e.stdout.decode()}\n\n{e.stderr.decode()}"

# Agents

In [18]:


from langgraph.prebuilt import create_react_agent


agent = create_react_agent(
    model="openai:gpt-5",
    tools=[compile_latex],
    prompt="You are an expert LaTeX programmer."
)

state = agent.invoke({
    "messages": """
        Using LaTeX beamer, create a set of lecture slides
        that teach the A* search algorithm to second-year
        computer science students in a Data Structures
        and Algorithms course.

        Use the LaTeX compilation tool to check if the code
        compiles before returning it.
        """
})

state

{'messages': [HumanMessage(content='\n        Using LaTeX beamer, create a set of lecture slides\n        that teach the A* search algorithm to second-year\n        computer science students in a Data Structures\n        and Algorithms course.\n\n        Use the LaTeX compilation tool to check if the code\n        compiles before returning it.\n        ', additional_kwargs={}, response_metadata={}, id='884ab09d-b7a2-46c3-a970-c20f0cca444a'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_mf8lsYLBnEgGQFxbzhegbGyy', 'function': {'arguments': '{"source":"\\\\documentclass{beamer}\\n\\\\usetheme{Madrid}\\n\\\\usecolortheme{seahorse}\\n\\n% Packages\\n\\\\usepackage{amsmath,amssymb}\\n\\\\usepackage{tikz}\\n\\\\usetikzlibrary{arrows.meta,positioning}\\n\\\\usepackage{algorithmicx}\\n\\\\usepackage{algpseudocode}\\n\\n% Title info\\n\\\\title[A* Search]{A* Search Algorithm}\\n\\\\subtitle{Data Structures and Algorithms}\\n\\\\author{Your Name}\\n\\\\institute{Departme

# Workflows

In [3]:
from typing_extensions import TypedDict
from langchain.chat_models import init_chat_model
from pydantic import BaseModel, Field
import operator

class Outline(BaseModel):
    slides: list[str] = Field(description=("A list where each item is a complete, detailed description "
            "for a single lecture slide. Each description should include "
            "all necessary content (e.g., text, bullet points, code, figure descriptions)."))

class State(TypedDict):
    topic: str
    outline: Outline
    latex_slides: Annotated[list[str], operator.add]
    latex: str
    compile_status: str
    compile_error: str


llm = init_chat_model("openai:gpt-4o")

In [4]:
# Outlining node

from pydantic import BaseModel, Field

def outline_slides(state: State):
    """
    Drafts an outline for the lecture slides,
    covering the most important topics.
    """

    structured_llm = llm.with_structured_output(Outline)
    outline = structured_llm.invoke(
        f"""
        I want to create a set of lecture slides to teach {state['topic']}.
    
        **CRITICAL INSTRUCTION:** Generate a separate, distinct item for each slide. 
        Do not combine multiple slides into a single item.

        Each slide should be fairly short (be mindful about how much
        should realistically fit on one slide). This content should
        be ready to turn into a slide, including any bullet points,
        paragraphs, code listings, or descriptions of figures.
        DO NOT INCLUDE IMAGES.

        Focus on only the content of the slide, not the style.
        """
    )

    return {"outline": outline}

In [5]:
class SlideWriterState(TypedDict):
    description: str
    latex_slides: Annotated[list[str], operator.add]

class LatexSlide(BaseModel):
    latex_slides: str = Field(description="LaTeX source code for the slide.")

def slide_writer(state: SlideWriterState):
    """
    Converts a description of a slide to
    LaTeX beamer source.
    """

    structured_llm = llm.with_structured_output(LatexSlide)
    msg = structured_llm.invoke(f"""
    Convert the following slide description to the LaTeX beamer
    source code (do not include the document boilerplate). Omit
    ALL images.
    
    {state['description']}
    """)

    return {"latex_slides": [msg.latex_slides]}

In [6]:
from langgraph.types import Send

def assign_slides(state: State):
    """
    Assign a worker for each slide.
    """
    return [Send("slide_writer", {"description": slide}) for slide in state["outline"].slides]

In [7]:
def combine_slides(state: State):
    """
    Combine all of the slides into one LaTeX beamer
    presentation.
    """
    latex = r"""
    \documentclass{beamer}

    % Common packages
    \usepackage[utf8]{inputenc}
    \usepackage[T1]{fontenc}
    \usetheme{Madrid}
    
    % Code highlighting setup
    \lstset{
        basicstyle=\ttfamily\footnotesize,
        keywordstyle=\color{blue},
        commentstyle=\color{green!60!black},
        stringstyle=\color{red},
        numbers=left,
        numberstyle=\tiny,
        breaklines=true,
        showstringspaces=false
    }
    \begin{document}
    """ + "\n\n".join(state['latex_slides']) + r"""
    \end{document}
    """

    return {"latex": latex}

In [8]:
def compile_check(state: State):
    print("compile check!")
    with tempfile.TemporaryDirectory() as working_dir:
        source_path = os.path.join(working_dir, "input.tex")
        with open(source_path, "w") as source_file:
            source_file.write(state['latex'])

        output_name = "output"

        try:
            subprocess.run(
                [
                    "pdflatex",
                    f"-jobname={output_name}",
                    "-interaction=nonstopmode",
                    source_path,
                ],
                cwd=working_dir,
                capture_output = True,
                check = True
            )
            print("Compiled successfully")
            return {"compile_status": "success", "compile_error": ""}
        except subprocess.CalledProcessError as e:
            print("Compilation failed.")
            return {"compile_status": "error", "compile_error": f"{e.stdout.decode()}\n\n{e.stderr.decode()}"}

In [14]:
class LatexSource(BaseModel):
    latex: str = Field(description="Complete LaTeX beamer source code file.")

def fix_compilation_error(state: State):
    structured_llm = llm.with_structured_output(LatexSource)
    
    msg = structured_llm.invoke(f"""
    There's an error compiling this LaTeX beamer
    source code file. Please make any necessary
    changes to ensure that this file compiles:

    {state['latex']}

    {state['compile_error']}
    """)

    print(msg.latex)

    return {"latex": msg.latex}

from langgraph.graph import START, END, StateGraph

workflow = StateGraph(State)

workflow.add_node("outline_slides", outline_slides)
workflow.add_node("slide_writer", slide_writer)
workflow.add_node("combine_slides", combine_slides)
workflow.add_node("compile_check", compile_check)
workflow.add_node("fix_compilation_error", fix_compilation_error)


workflow.add_edge(START, "outline_slides")

workflow.add_conditional_edges("outline_slides", assign_slides, ["slide_writer"])

workflow.add_edge("slide_writer", "combine_slides")
workflow.add_edge("combine_slides", "compile_check")

workflow.add_conditional_edges("compile_check", lambda s: s["compile_status"], {
    "success": END,
    "error": END,
})

# workflow.add_edge("fix_compilation_error", "compile_check")

chain = workflow.compile()
state = chain.invoke({"topic": "A* search"})
state

compile check!
Compilation failed.


{'topic': 'A* search',
 'outline': Outline(slides=['A* Search: Overview and Learning Goals\n- What problems A* solves: optimal pathfinding in weighted graphs\n- Where it’s used: robotics, games, routing, planning\n- Goals today:\n  - Understand g, h, and f = g + h\n  - Know when A* is optimal and complete\n  - Implement graph-search A* with a priority queue\n  - Design effective heuristics\n  - Recognize variants (Weighted A*, IDA*)', "Formulating a Search Problem\n- State space: set of reachable states\n- Actions and transitions: s -> s' with step cost c(s, a, s') ≥ 0\n- Start state(s) and goal test\n- Path cost: sum of step costs along a path\n- Objective: find a lowest-cost path from start to a goal", 'Recap: Uniform-Cost Search (UCS)\n- Expands node with smallest g(n) (cost from start)\n- Optimal and complete when step costs ≥ 0\n- Weakness: no guidance toward the goal\n- Motivation for heuristics: estimate remaining cost to focus search', 'Heuristics and Best-First Idea\n- Heurist

In [16]:
state['latex']

"\n    \\documentclass{beamer}\n\n    % Common packages\n    \\usepackage[utf8]{inputenc}\n    \\usepackage[T1]{fontenc}\n    \\usetheme{Madrid}\n\n    % Code highlighting setup\n    \\lstset{\n        basicstyle=\\ttfamily\\footnotesize,\n        keywordstyle=\\color{blue},\n        commentstyle=\\color{green!60!black},\n        stringstyle=\\color{red},\n        numbers=left,\n        numberstyle=\\tiny,\n        breaklines=true,\n        showstringspaces=false\n    }\n    \\begin{document}\n    \\begin{frame}{A* Search: Overview and Learning Goals}\n\\begin{itemize}\n  \\item What problems A* solves: optimal pathfinding in weighted graphs\n  \\item Where it's used: robotics, games, routing, planning\n  \\item Goals today:\n  \\begin{itemize}\n    \\item Understand $g$, $h$, and $f = g + h$\n    \\item Know when A* is optimal and complete\n    \\item Implement graph-search A* with a priority queue\n    \\item Design effective heuristics\n    \\item Recognize variants (Weighted A*, ID